In [9]:
import os
import glob
from time import time

import numpy as np

from sklearn.linear_model import RidgeClassifier

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from torch.utils.data.dataset import Dataset
from torchvision.datasets.folder import default_loader

from tlopu.model_utils import pick_model, cut_model, get_model_size, get_output_size
from tlopu.features import fast_conv_features, decoding, get_random_features, dummy_predict_GPU
from tlopu.dataset import CatsAndDogs, CUB_200, Animals10

In [2]:
def get_mean_std(train_loader):
    mean, std = torch.zeros(3), torch.zeros(3)

    for batch_id, (image, target) in enumerate(train_loader):
        mean += torch.mean(image, dim=(0,2,3))
        std += torch.std(image, dim=(0,2,3))

    mean = mean / len(train_loader)
    std = std / len(train_loader)
    
    return mean, std

In [62]:
def get_loaders(dataset_name, batch_size=32, num_workers=12, mean=None, std=None):
    
    transform_list = [transforms.Resize((224, 224)), transforms.ToTensor()]
    if mean is not None:
        transform_list.append(transforms.Normalize(mean=mean, std=std))
    data_transform = transforms.Compose(transform_list)
    
    if dataset_name == "cats_dogs":
        dataset_path = "/data/home/luca/datasets/cats-and-dogs-breeds-classification-oxford-dataset"
        
        train_dataset = CatsAndDogs(dataset_path, mode="trainval", transform=data_transform)
        test_dataset = CatsAndDogs(dataset_path, mode="test", transform=data_transform)
    
    elif dataset_name == "CUB_200":
        dataset_path = "/data/home/luca/datasets/CUB_200_2011/"
        
        train_dataset = CUB_200(dataset_path, mode="train", transform=data_transform)
        test_dataset = CUB_200(dataset_path, mode="test", transform=data_transform)
    
    elif dataset_name == "animals10":
        path = "/data/home/luca/datasets/animals10/raw-img/"
        train_dataset = Animals10(path, test_ratio=20, mode="train", transform=data_transform)
        test_dataset = Animals10(path, test_ratio=20, mode="test", transform=data_transform)
        
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    
    return train_loader, test_loader

In [63]:
train_loader, test_loader = get_loaders("animals10", batch_size=32, num_workers=12)
print("Computing dataset mean...")
mean, std = get_mean_std(train_loader)
train_loader, test_loader = get_loaders("animals10", batch_size=32, num_workers=12, mean=mean, std=std)


Computing dataset mean...


In [64]:
model_name, model_options = "resnet18", "noavgpool"
device = "cuda:0"
encode_type = "positive"
model_dtype = "float32"

model, output_size = pick_model(model_name=model_name, model_options=model_options, device=device, dtype=model_dtype)

resnet18 model loaded successfully.


In [66]:
enc_train_features, train_labels, train_conv_time, train_encode_time = fast_conv_features(train_loader, model,
                                                                                          output_size,
                                                                                          device=device,
                                                                                          encode_type=encode_type,
                                                                                          dtype=model_dtype)
print("{0} - train conv features time = {1:3.2f} s\tencoding = {2:1.5f} s\tshape = {3}"
      .format(model_name, train_conv_time, train_encode_time, enc_train_features.shape))

enc_test_features, test_labels, test_conv_time, test_encode_time = fast_conv_features(test_loader, model,
                                                                                      output_size, device=device,
                                                                                      encode_type=encode_type,
                                                                                      dtype=model_dtype)
print("{0} - test conv features time  = {1:3.2f} s\tencoding = {2:1.5f} s\tshape = {3}"
      .format(model_name, test_conv_time, test_encode_time, enc_test_features.shape))

resnet18 - train conv features time = 12.40 s	encoding = 0.00903 s	shape = (19363, 25088)
resnet18 - test conv features time  = 3.91 s	encoding = 0.00217 s	shape = (4846, 25088)


In [67]:
n_components = 0
train_proj_time, train_random_features = get_random_features(enc_train_features, n_components)
test_proj_time, test_random_features = get_random_features(enc_test_features, n_components)

In [68]:
train_decode_time, dec_train_random_features = decoding(train_random_features, decode_type=None)
test_decode_time, dec_test_random_features = decoding(test_random_features, decode_type=None)

In [2]:
15*100/60

25.0

In [69]:
alpha_exp_min, alpha_exp_max, alpha_space = 2, 8, 2

alpha_mant = np.linspace(1, 9, alpha_space)
alphas = np.concatenate([alpha_mant * 10 ** i for i in range(alpha_exp_min, alpha_exp_max + 1)])

for alpha in alphas:
    clf = RidgeClassifier(alpha=alpha)
    since = time()
    clf.fit(dec_train_random_features, train_labels)
    fit_time = time() - since
    
    
    train_accuracy = clf.score(dec_train_random_features, train_labels) * 100
    test_accuracy = clf.score(dec_test_random_features, test_labels) * 100
    
    print("alpha = {0:.2e}\tTrain acc = {1:2.2f}\tTest acc = {2:2.2f}".format(alpha, train_accuracy, test_accuracy))

alpha = 1.00e+02	Train acc = 100.00	Test acc = 89.66


KeyboardInterrupt: 